In [ ]:
import tensorflow as tf

from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

In [ ]:
from numpy import genfromtxt
import pandas as pd
x = genfromtxt('/content/drive/MyDrive/Colab Notebooks/WISDM_x.csv', delimiter=',')
y = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/WISDM_y.csv').to_numpy()

In [ ]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
le = LabelEncoder()
y = le.fit_transform(y).reshape(-1,1)


ohe = OneHotEncoder()
y = ohe.fit_transform(y).toarray()

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
def create_series(x,y,timestep,overlap):

  slide_step = int(timestep*(1-overlap))
  data_num = int((len(x)/slide_step)-1)
  
  dataset = np.ndarray(shape=(data_num,timestep,len(x[0])))
  labels = list()

  for i in range(data_num):
    labels.append(y[slide_step*(i+1)-1])
    for j in range(timestep):
      dataset[i,j,:] = x[slide_step*i+j,:]

  return dataset,np.array(labels)


In [ ]:
# y_pred.append("5")
import numpy as np
dataset,labels = create_series(x,y,16,0.5)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset,labels,test_size=0.3, random_state=412)
# X_test, X_val, y_test, y_val = train_test_split(X_toSplit,y_toSplit,test_size=0.25, random_state=412)

In [ ]:
labels.shape

(131070, 6)

In [ ]:
# Input Data 

training_data_count = len(X_train)  # 96092 training series (with 50% overlap between each serie)
test_data_count = len(X_test)  # 41183 testing series
n_steps = len(X_train[0])  # 16 timesteps per series
n_input = len(X_train[0][0])  # 3 input parameters per timestep
print(n_steps,n_input)

# LSTM Neural Network's internal structure
# n_hidden = 32 # Hidden layer num of features
n_classes = 6 # Total classes (should go up, or should go down)


# Training 

learning_rate = 0.0025
lambda_loss_amount = 0.0015
training_iters = training_data_count * 300  # Loop 300 times on the dataset
batch_size = 64
display_iter = 30000  # To show test set accuracy during training


# Some debugging info

print("Some useful info to get an insight on dataset's shape and normalisation:")
print("(X shape, y shape, every X's mean, every X's standard deviation)")
print(X_test.shape, y_test.shape, np.mean(X_test), np.std(X_test))
print("The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.")

16 3
Some useful info to get an insight on dataset's shape and normalisation:
(X shape, y shape, every X's mean, every X's standard deviation)
(39321, 16, 3) (39321, 6) 2.873419266367973 6.908464983446615
The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.


In [ ]:
model = models.Sequential()
model.add(layers.Conv1D(256, 3, input_shape=(n_steps, n_input)))
model.add(layers.Dropout(0.2))

model.add(layers.Conv1D(128,3))
model.add(layers.Dropout(0.2))

model.add(layers.Conv1D(64,3))
model.add(layers.Dropout(0.2))

model.add(layers.Conv1D(32,3))
model.add(layers.Dropout(0.2))

model.add(layers.Flatten())
model.add(layers.Dense(6, activation='softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])

model.fit(X_train,y_train,batch_size=64,epochs=300,validation_split=0.1)

model.save('/cnn_model')

Epoch 1/300
1291/1291 [==============================] - 15s 5ms/step - loss: 1.4125 - accuracy: 0.5007 - val_loss: 1.2330 - val_accuracy: 0.5457
Epoch 2/300
1291/1291 [==============================] - 6s 4ms/step - loss: 1.2670 - accuracy: 0.5355 - val_loss: 1.2241 - val_accuracy: 0.5454
Epoch 3/300
1291/1291 [==============================] - 6s 5ms/step - loss: 1.2637 - accuracy: 0.5374 - val_loss: 1.2342 - val_accuracy: 0.5520
Epoch 4/300
1291/1291 [==============================] - 6s 5ms/step - loss: 1.2625 - accuracy: 0.5377 - val_loss: 1.2380 - val_accuracy: 0.5458
Epoch 5/300
1291/1291 [==============================] - 6s 4ms/step - loss: 1.2614 - accuracy: 0.5371 - val_loss: 1.2268 - val_accuracy: 0.5442
Epoch 6/300
1291/1291 [==============================] - 6s 4ms/step - loss: 1.2587 - accuracy: 0.5396 - val_loss: 1.2356 - val_accuracy: 0.5548
Epoch 7/300
1291/1291 [==============================] - 6s 5ms/step - loss: 1.2576 - accuracy: 0.5397 - val_loss: 1.2229 - val_a

KeyboardInterrupt: ignored